# Iris 꽃 분류 딥러닝 모델

## 🌸 프로젝트 개요
이 노트북에서는 Scikit-learn의 Iris 데이터셋을 사용하여 꽃의 품종(Setosa, Versicolor, Virginica)을 분류하는 딥러닝 모델을 구현합니다. 소량의 정형 데이터에 딥러닝을 적용하는 과정을 통해, 데이터 전처리의 중요성과 모델 설계 시 고려사항을 학습합니다.

## 📚 학습 목표
- Scikit-learn 데이터셋을 딥러닝 모델에 활용하는 방법 학습
- 수치형 데이터에 대한 표준화(`StandardScaler`)의 필요성 및 적용 방법 이해
- 다중 분류 문제에서 원-핫 인코딩(`One-Hot Encoding`)의 역할과 구현 방법 습득
- 작은 데이터셋에 대한 과적합을 피하기 위한 간단한 딥러닝 모델 설계
- `categorical_crossentropy` 손실 함수 사용법 이해

## 1. 라이브러리 임포트 및 환경 설정

딥러닝 모델 구현과 데이터 전처리에 필요한 라이브러리를 임포트하고, 실험의 재현성을 보장하기 위해 랜덤 시드를 설정합니다.

In [1]:
# 데이터 처리 및 분할 라이브러리
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler

# 딥러닝 라이브러리
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical

# 재현 가능한 결과를 위한 랜덤 시드 설정
tf.random.set_seed(1)

print("TensorFlow 버전:", tf.__version__)
print("✅ 라이브러리 임포트 완료!")

TensorFlow 버전: 2.19.0
✅ 라이브러리 임포트 완료!


## 2. Iris 데이터셋 로드 및 탐색

Scikit-learn에 내장된 `load_iris()` 함수를 사용하여 Iris 데이터셋을 로드하고, 데이터의 구조와 특성을 확인합니다.

In [2]:
# Iris 데이터셋 로드
iris = load_iris()

# 특성(X)과 타겟(y) 분리
X = iris['data']      # 꽃의 특성 데이터 (4개 특성)
y = iris['target']    # 꽃의 품종 라벨 (3개 클래스)

# 데이터셋 기본 정보 확인
print("📊 Iris 데이터셋 기본 정보:")
print(f"특성 데이터 형태: {X.shape}")
print(f"타겟 데이터 형태: {y.shape}")
print(f"특성 이름: {iris.feature_names}")
print(f"클래스 이름: {iris.target_names}")

print(f"\n📈 클래스별 분포:")
import numpy as np
unique, counts = np.unique(y, return_counts=True)
for i, (cls, count) in enumerate(zip(iris.target_names, counts)):
    print(f"   {i}: {cls} - {count}개")

print(f"\n✅ 데이터셋 로드 완료! (총 {len(X)}개 샘플)")

📊 Iris 데이터셋 기본 정보:
특성 데이터 형태: (150, 4)
타겟 데이터 형태: (150,)
특성 이름: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
클래스 이름: ['setosa' 'versicolor' 'virginica']

📈 클래스별 분포:
   0: setosa - 50개
   1: versicolor - 50개
   2: virginica - 50개

✅ 데이터셋 로드 완료! (총 150개 샘플)


## 3. 데이터 분할 (훈련/테스트)

모델의 일반화 성능을 객관적으로 평가하기 위해, `train_test_split` 함수를 사용하여 전체 데이터를 훈련셋과 테스트셋으로 분할합니다.

In [3]:
# 데이터를 훈련셋과 테스트셋으로 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    random_state=1,    # 재현 가능한 결과를 위한 시드
    test_size=0.3      # 테스트셋 비율 30%
)

# 분할 결과 확인
print("📊 데이터 분할 결과:")
print(f"훈련 특성 데이터: {X_train.shape}")
print(f"훈련 타겟 데이터: {y_train.shape}")
print(f"테스트 특성 데이터: {X_test.shape}")
print(f"테스트 타겟 데이터: {y_test.shape}")

print(f"\n📈 분할 비율:")
print(f"훈련셋: {len(X_train)}개 ({len(X_train)/len(X)*100:.1f}%)")
print(f"테스트셋: {len(X_test)}개 ({len(X_test)/len(X)*100:.1f}%)")

print("\n✅ 데이터 분할 완료!")

📊 데이터 분할 결과:
훈련 특성 데이터: (105, 4)
훈련 타겟 데이터: (105,)
테스트 특성 데이터: (45, 4)
테스트 타겟 데이터: (45,)

📈 분할 비율:
훈련셋: 105개 (70.0%)
테스트셋: 45개 (30.0%)

✅ 데이터 분할 완료!


## 4. 특성 데이터 정규화 (StandardScaler)

딥러닝 모델의 학습 효율성과 성능을 높이기 위해, 각 특성의 스케일이 다른 수치형 데이터를 표준화합니다.

### 정규화가 필요한 이유:
- **학습 안정성**: 각 특성의 스케일이 다르면 경사하강법 기반의 최적화 과정이 불안정해질 수 있습니다.
- **수렴 속도 향상**: 모든 특성을 비슷한 스케일로 맞추면 손실 함수의 등고선이 원형에 가까워져, 옵티마이저가 최적점을 더 빠르고 쉽게 찾을 수 있습니다.
- **편향 방지**: 스케일이 큰 특정 변수가 모델 학습에 과도한 영향을 미치는 것을 방지합니다.

⚠️ **중요**: `StandardScaler`는 **훈련 데이터로만 학습(`fit`)**하고, 그 학습된 스케일러를 사용하여 훈련 데이터와 테스트 데이터를 모두 변환(`transform`)해야 합니다. 이는 테스트 데이터의 정보가 훈련 과정에 유출(Data Leakage)되는 것을 방지하기 위함입니다.

In [4]:
# 정규화 전 데이터 범위 확인
print("정규화 전 데이터 범위:")
print(f"X_train 최솟값: {X_train.min(axis=0)}")
print(f"X_train 최댓값: {X_train.max(axis=0)}")

# StandardScaler를 사용한 정규화
scaler = StandardScaler()

# 훈련 데이터로 스케일러를 학습하고 변환
X_train_scaled = scaler.fit_transform(X_train)

# 테스트 데이터는 훈련 데이터의 스케일로 변환 (fit 없이 transform만)
X_test_scaled = scaler.transform(X_test)

# 정규화 후 데이터 범위 확인
print(f"\n정규화 후 데이터 범위:")
print(f"X_train_scaled 평균: {X_train_scaled.mean(axis=0).round(3)}")
print(f"X_train_scaled 표준편차: {X_train_scaled.std(axis=0).round(3)}")

print(f"\n📊 데이터 형태 확인:")
print(f"훈련 데이터: {X_train_scaled.shape}")
print(f"테스트 데이터: {X_test_scaled.shape}")

print("\n✅ 특성 데이터 정규화 완료!")
print("📝 StandardScaler: 평균=0, 표준편차=1로 정규화")

정규화 전 데이터 범위:
X_train 최솟값: [4.3 2.  1.  0.1]
X_train 최댓값: [7.7 4.4 6.9 2.5]

정규화 후 데이터 범위:
X_train_scaled 평균: [-0. -0.  0. -0.]
X_train_scaled 표준편차: [1. 1. 1. 1.]

📊 데이터 형태 확인:
훈련 데이터: (105, 4)
테스트 데이터: (45, 4)

✅ 특성 데이터 정규화 완료!
📝 StandardScaler: 평균=0, 표준편차=1로 정규화


## 5. 타겟 데이터 원-핫 인코딩

딥러닝 다중 분류 모델의 출력층은 각 클래스에 대한 확률을 출력하므로, 타겟 라벨을 이에 맞는 형태로 변환해야 합니다. 원-핫 인코딩은 이러한 변환을 수행하는 가장 일반적인 방법입니다.

### 원-핫 인코딩이란?
- **정수 라벨**: `[0, 1, 2]`
- **원-핫 인코딩**: `[[1,0,0], [0,1,0], [0,0,1]]`
- `categorical_crossentropy` 손실 함수는 원-핫 인코딩된 라벨을 입력으로 기대합니다.

In [5]:
# 원-핫 인코딩 전 라벨 확인
print("원-핫 인코딩 전:")
print(f"y_train 형태: {y_train.shape}")
print(f"y_train 샘플: {y_train[:10]}")
print(f"고유값: {np.unique(y_train)}")

# 타겟 라벨을 원-핫 인코딩으로 변환
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

# 원-핫 인코딩 후 라벨 확인
print(f"\n원-핫 인코딩 후:")
print(f"y_train_encoded 형태: {y_train_encoded.shape}")
print(f"y_train_encoded 샘플:")
print(y_train_encoded[:5])

print(f"\n📊 변환 결과:")
print(f"훈련 라벨: {y_train_encoded.shape}")
print(f"테스트 라벨: {y_test_encoded.shape}")

# 변환 확인 예시
print(f"\n📝 변환 예시:")
for i in range(3):
    original = np.where(y_train == i)[0][0]  # 각 클래스의 첫 번째 샘플 인덱스
    print(f"원본 라벨 {i} → 원-핫: {y_train_encoded[original]}")

print("\n✅ 원-핫 인코딩 완료!")

원-핫 인코딩 전:
y_train 형태: (105,)
y_train 샘플: [2 0 0 0 1 0 0 2 2 2]
고유값: [0 1 2]

원-핫 인코딩 후:
y_train_encoded 형태: (105, 3)
y_train_encoded 샘플:
[[0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]

📊 변환 결과:
훈련 라벨: (105, 3)
테스트 라벨: (45, 3)

📝 변환 예시:
원본 라벨 0 → 원-핫: [1. 0. 0.]
원본 라벨 1 → 원-핫: [0. 1. 0.]
원본 라벨 2 → 원-핫: [0. 0. 1.]

✅ 원-핫 인코딩 완료!


## 6. 딥러닝 모델 구조 설계

Iris 데이터셋 분류를 위한 다층 퍼셉트론 모델을 설계합니다. 데이터셋이 작으므로, 과적합을 방지하기 위해 비교적 간단한 네트워크 구조를 사용합니다.

### 모델 구조:
- **입력층**: 4개 뉴런 (4개 특성)
- **은닉층 1**: 64개 뉴런, `ReLU` 활성화 함수
- **은닉층 2**: 64개 뉴런, `ReLU` 활성화 함수
- **은닉층 3**: 128개 뉴런, `ReLU` 활성화 함수
- **출력층**: 3개 뉴런, `Softmax` 활성화 함수 (3개 클래스에 대한 확률 출력)

In [6]:
# Sequential 모델을 사용하여 신경망 구조 설계
network = keras.models.Sequential([
    # 입력층을 별도로 정의하지 않고, 첫 번째 Dense 층에 input_shape를 지정
    keras.layers.Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    
    # 은닉층 2: 64개 뉴런, ReLU 활성화 함수
    keras.layers.Dense(64, activation='relu'),
    
    # 은닉층 3: 128개 뉴런, ReLU 활성화 함수
    keras.layers.Dense(128, activation='relu'),
    
    # 출력층: 3개 뉴런 (3개 클래스), Softmax 활성화 함수
    keras.layers.Dense(3, activation='softmax')
])

# 모델 요약 정보 출력
print("🏗️ 모델 구조:")
network.summary()

print("\n✅ 신경망 모델 구조 설계 완료!")
print("\n📝 모델 설계 포인트:")
print("- 작은 데이터셋이므로 과적합을 방지하기 위해 적당한 크기의 네트워크 사용")
print("- ReLU 활성화 함수로 학습 효율성 증대")
print("- Softmax 출력층으로 다중분류 확률 출력")

🏗️ 모델 구조:


<pre>Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                320       
                                                                 
 dense_1 (Dense)             (None, 64)                4160      
                                                                 
 dense_2 (Dense)             (None, 128)               8320      
                                                                 
 dense_3 (Dense)             (None, 3)                 387       
                                                                 
Total params: 13,187 (51.51 KB)
Trainable params: 13,187 (51.51 KB)
Non-trainable params: 0 (0.00 B)
_________________________________________________________________
</pre>


✅ 신경망 모델 구조 설계 완료!

📝 모델 설계 포인트:
- 작은 데이터셋이므로 과적합을 방지하기 위해 적당한 크기의 네트워크 사용
- ReLU 활성화 함수로 학습 효율성 증대
- Softmax 출력층으로 다중분류 확률 출력


## 7. 모델 컴파일 설정

모델을 학습할 수 있도록 옵티마이저, 손실 함수, 평가지표를 설정합니다.

### 설정 요소:
- **옵티마이저**: `rmsprop`
- **손실함수**: `categorical_crossentropy` (원-핫 인코딩된 다중분류용)
- **평가지표**: `accuracy` (정확도)

In [7]:
# 모델 컴파일 설정
network.compile(
    optimizer='rmsprop',                  # 옵티마이저: RMSprop
    loss='categorical_crossentropy',      # 손실함수: 원-핫 인코딩된 다중분류용
    metrics=['accuracy']                  # 평가지표: 정확도
)

print("✅ 모델 컴파일 완료!")
print("\n📝 손실함수 비교:")
print("- categorical_crossentropy: 라벨이 원-핫 인코딩 형태 ([1,0,0], [0,1,0], ...)")
print("- sparse_categorical_crossentropy: 라벨이 정수 형태 (0, 1, 2, ...)")
print("  → 현재는 라벨을 원-핫 인코딩했으므로 categorical_crossentropy 사용")

print(f"\n🎯 모델 학습 준비 완료!")
print(f"입력 데이터: {X_train_scaled.shape}")
print(f"출력 데이터: {y_train_encoded.shape}")

✅ 모델 컴파일 완료!

📝 손실함수 비교:
- categorical_crossentropy: 라벨이 원-핫 인코딩 형태 ([1,0,0], [0,1,0], ...)
- sparse_categorical_crossentropy: 라벨이 정수 형태 (0, 1, 2, ...)
  → 현재는 라벨을 원-핫 인코딩했으므로 categorical_crossentropy 사용

🎯 모델 학습 준비 완료!
입력 데이터: (105, 4)
출력 데이터: (105, 3)


## 8. 모델 학습 실행

`fit()` 함수를 사용하여 모델을 학습시킵니다.

### 학습 매개변수:
- **epochs**: 30 (전체 데이터셋을 30번 반복 학습)
- **batch_size**: 100 (한 번에 100개 샘플씩 처리)

In [8]:
# 모델 학습 실행
print("🚀 모델 학습을 시작합니다...")
print("⏰ Iris 데이터셋은 작으므로 빠르게 학습됩니다.")

history = network.fit(
    X_train_scaled,      # X: 정규화된 훈련 특성 데이터
    y_train_encoded,     # y: 원-핫 인코딩된 훈련 라벨
    epochs=30,           # 학습 회수 (30 에포크)
    batch_size=100,      # 배치 크기 (전체 데이터보다 큰 값이므로 실제로는 전체 배치)
    verbose=1            # 학습 진행 상황 출력
)

print("\n✅ 모델 학습 완료!")
print("\n📝 배치 크기 참고사항:")
print(f"- 설정된 배치 크기: 100")
print(f"- 실제 훈련 데이터 크기: {len(X_train_scaled)}")
print("- 훈련 데이터가 배치 크기보다 작으므로 실제로는 전체 배치로 학습됨")

🚀 모델 학습을 시작합니다...
⏰ Iris 데이터셋은 작으므로 빠르게 학습됩니다.
Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.0578 - loss: 1.1517 
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.5613 - loss: 1.0150
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.6063 - loss: 0.9203
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.6838 - loss: 0.8377
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7806 - loss: 0.7653
Epoch 6/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.8871 - loss: 0.7038
Epoch 7/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.9419 - loss: 0.6502
Epoch 8/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9419 - loss: 0.6035
Epoch 9/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9225 - loss: 0.5602
Epoch 10/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - accuracy: 0.9129 - loss: 0.5205
Epoch 11/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9225 - loss: 0.4840
Epoch 12/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 

## 9. 모델 성능 평가

`evaluate()` 함수를 사용하여 훈련셋과 테스트셋에서의 모델 성능을 평가하고, 과적합 여부를 진단합니다.

In [9]:
# 훈련셋 성능 평가
train_loss, train_acc = network.evaluate(X_train_scaled, y_train_encoded, verbose=0)
print("📊 훈련셋 성능:")
print(f"   손실(Loss): {train_loss:.4f}")
print(f"   정확도(Accuracy): {train_acc:.4f} ({train_acc*100:.2f}%)")

# 테스트셋 성능 평가
test_loss, test_acc = network.evaluate(X_test_scaled, y_test_encoded, verbose=0)
print("\n📊 테스트셋 성능:")
print(f"   손실(Loss): {test_loss:.4f}")
print(f"   정확도(Accuracy): {test_acc:.4f} ({test_acc*100:.2f}%)")

# 성능 차이 분석
accuracy_diff = train_acc - test_acc
print(f"\n📈 성능 분석:")
print(f"   정확도 차이: {accuracy_diff:.4f} ({accuracy_diff*100:.2f}%p)")

if accuracy_diff > 0.1:  # 10% 이상 차이
    print("   ⚠️  과적합(Overfitting) 가능성이 있습니다.")
elif accuracy_diff < -0.05:  # 테스트가 훈련보다 5% 이상 좋음
    print("   🤔 테스트셋 성능이 더 좋습니다. 데이터 분할을 다시 확인해보세요.")
else:
    print("   ✅ 적절한 일반화 성능을 보입니다.")

print(f"\n🎉 Iris 꽃 분류 딥러닝 모델 구현 완료!")
print(f"📊 최종 테스트 정확도: {test_acc*100:.1f}%")

📊 훈련셋 성능:
   손실(Loss): 0.1430
   정확도(Accuracy): 0.9524 (95.24%)

📊 테스트셋 성능:
   손실(Loss): 0.2610
   정확도(Accuracy): 0.8667 (86.67%)

📈 성능 분석:
   정확도 차이: 0.0857 (8.57%p)
   ✅ 적절한 일반화 성능을 보입니다.

🎉 Iris 꽃 분류 딥러닝 모델 구현 완료!
📊 최종 테스트 정확도: 86.7%


## 10. 학습 요약 및 MNIST와의 비교

### ✅ Iris 딥러닝 구현 내용:
1. **Scikit-learn 데이터셋 로드**: `load_iris`를 사용하여 정형 데이터를 로드했습니다.
2. **데이터 분할 및 정규화**: `train_test_split`으로 데이터를 분할하고, `StandardScaler`를 사용하여 특성 스케일을 통일했습니다.
3. **원-핫 인코딩**: 다중 분류 문제에 맞게 정수 라벨을 원-핫 벡터로 변환했습니다.
4. **소규모 데이터셋용 모델 설계**: 과적합을 피하기 위해 비교적 간단한 MLP 모델을 구축했습니다.
5. **모델 학습 및 평가**: `categorical_crossentropy` 손실 함수를 사용하여 모델을 학습하고, 훈련 및 테스트 성능을 비교 분석했습니다.

### 🔄 MNIST와 Iris의 차이점 및 핵심 학습 포인트:

| 구분 | MNIST (대용량 이미지) | Iris (소량 정형 데이터) |
|:---|:---|:---|| **데이터 유형** | 비정형 데이터 (이미지) | 정형 데이터 (수치) |
| **데이터 크기** | 70,000개 | 150개 |
| **주요 전처리** | 차원 변환, 0-1 스케일링 | 표준화 (평균 0, 분산 1) |
| **라벨/손실함수** | 정수 / `sparse_categorical_crossentropy` | 원-핫 / `categorical_crossentropy` |
| **모델 복잡도** | 더 깊고 복잡한 모델 가능 (예: CNN) | 간단한 모델로 시작 (과적합 방지) |
| **적합한 기법** | **딥러닝**이 강력한 성능 발휘 | **전통적 머신러닝**도 매우 효과적 |

이번 실습을 통해 데이터의 특성(크기, 유형, 구조)에 따라 전처리 방법과 모델 설계 전략이 어떻게 달라져야 하는지 명확히 이해할 수 있었습니다.